In [1]:
%pylab inline
import pandas as pd
import datetime
import ipywidgets
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
xls = pd.ExcelFile('MPAN Data - interim 080819.xlsx')

In [3]:
sheet_to_df_map = {}
for sheet in xls.sheet_names:
    sheet_to_df_map[sheet] = xls.parse(sheet)

In [189]:
# in this case we only need one sheet
df = sheet_to_df_map['2018 2019']
df.columns = df.iloc[0] #fixing header
df = df.drop(df.index[0])
df.head()
df.shape

(2909, 589)

In [5]:

'''
counting from 0 the 9th column is the first
with reading data, lets find this with a 
script:
'''
for i in range(len(df.columns.tolist())):
    if type(df.columns.tolist()[i]) == datetime.datetime:
        fromHere = i
        break
print(fromHere)
shpe = df.shape

print(shpe[0], shpe[1])

pb = ipywidgets.IntProgress(max = shpe[0] - 1)
display(pb)

indicesOfEmptyRows = []
index = 0
for j in range(shpe[0]):
    if df.iloc[j,fromHere:].isnull().values.all():
        indicesOfEmptyRows.append(j)
    pb.value = index
    index += 1
df2 = df.drop(indicesOfEmptyRows, inplace=False)
df2.to_excel("CleandFromAllNull.xlsx")

9
2909 589


A Jupyter Widget

In [123]:
df2 = pd.read_excel("CleandFromAllNull.xlsx")
df = df2.reset_index(drop=True)

In [190]:
indicesWhereNotEnoughDataPoints = []
pb = ipywidgets.IntProgress(max = df.shape[0] - 1)
display(pb)
ind = 0

for i in range(df.shape[0]):
    pb.value = ind
    ind += 1
    whereIsData = (np.where(df.iloc[i,fromHere:].isnull().values == False))[0] # returns array of indices of real datapoints
    data = df.iloc[i,fromHere:].values
    #print(len(whereIsData), len(data))
    #print(f'whereIsData: {whereIsData} \n data: {data[whereIsData]}')
    #print('----')
    if len(whereIsData) < 2:
        indicesWhereNotEnoughDataPoints.append(i)
    else:
        ok = False
        begin = whereIsData[0]
        beginHasFisrtValue = True
        if begin > 0:
            for k in range(begin):
                data[k] = 0
        for j in range(1, len(whereIsData)):

            end = whereIsData[j]
            
            value = (data[end]-data[begin])/(end-begin)
            #print(f'j: {j}, end: {end}, value: {value}')
            if value < 0:
                for k in range(begin, end):
                    data[k] = 0
            else:
                if beginHasFisrtValue:
                    data[begin] = value
                    beginHasFisrtValue = False
                else:
                    data[begin] = (data[begin] + value) / 2
                for k in range(begin+1, end+1):
                    data[k] = value
            begin = end
        for j in range(end, len(data)):
            data[j] = 0
        #print(f'Final data: {data}')
    df = changeValues(df, data, i, arange(fromHere, df.shape[1]))
df2 = df.drop(indicesWhereNotEnoughDataPoints, inplace = False)

A Jupyter Widget

In [196]:
df2.to_csv("Interpolated.csv")

In [19]:
numberOfDataPoints = len(df.iloc[i,fromHere:])-sum(df.iloc[0,fromHere:].isnull().values)

579

In [33]:
whereIsData = np.where(df.iloc[0,fromHere:].isnull().values == False) # returns array of indices of real datapoints

In [167]:
df.head()

,First Read,Last Read,Advance,First Read,Last Read,Days,Annual Advance,Readings,MPAN ID,2018-01-01 00:00:00,...,2019-07-25 00:00:00,2019-07-26 00:00:00,2019-07-27 00:00:00,2019-07-28 00:00:00,2019-07-29 00:00:00,2019-07-30 00:00:00,2019-07-31 00:00:00,2019-08-01 00:00:00,2019-08-02 00:00:00,2019-08-03 00:00:00
1,1752911,1752911,0,00:00:00,00:00:00,0,,1,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1375558,1657284,281726,00:00:00,00:00:00,00:00:00,,19,9,1375558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,00:00:00,00:00:00,0,,0,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1136382,1223559,87177,00:00:00,00:00:00,0,,18,13,1136382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,0,00:00:00,00:00:00,0,,0,2806,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
whereIsData = (np.where(df.iloc[1,fromHere:].isnull().values == False))[0]
df.iloc[1, 8:10]

0
MPAN ID                      9
2018-01-01 00:00:00    1375558
Name: 2, dtype: object

In [112]:
foo = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5], 'c':[6,7,8]})
print(type(foo))
print(type(1))
#foo.loc[1, 'b'] = 9
foo

<class 'pandas.core.frame.DataFrame'>
<class 'int'>


,a,b,c
0,1,3,6
1,2,4,7
2,3,5,8


In [114]:
def changeValues(df, mylist, row, indices):
    '''
    @summary: changes a set of values within DataFrame's row to values from parameter mylist
    @param df: DataFrame in which we make the changes
    @type df: pandas.core.frame.DataFrame
    @param mylist: list with values to change DataFrame's elements
    @type mylist: list
    @param row: the index of row in which we make the changes
    @type row: int
    @param indices: the number indices of columns in which we make changes
    @type indices: list
    '''
    columnNames = df.columns
    if len(mylist) != len(indices):
        print("Error! len(mylist) != len(indices)")
        return 0
    for i in range(len(indices)):
        df.loc[row, columnNames[indices[i]]] = mylist[i]
    return df

In [166]:
df.loc[1,df.columns[420]]

nan

In [154]:
df[0, str(df.columns[31])]

KeyError: (0, '2018-01-23 00:00:00')